replicate titanic example with 1 set of weights nx1
write it with torch autograd
gd manual


In [1]:
import pandas as pd
import numpy as np
import torch

torch.manual_seed(0)
torch.autograd.set_detect_anomaly(True)

In [2]:
df = pd.read_csv("titanic.csv")
df = df.dropna()

df.drop(["PassengerId", "Name", "Cabin"], axis=1, inplace=True)
# create Class1 col and Class2 col where class1 = 1 iff Pclass == 1; if Pclass == 2 Class1 = 0, Class2 = 1;
df["Class1"] = 0
df.loc[df["Pclass"] == 1, "Class1"] = 1

df["Class2"] = 0
df.loc[df["Pclass"] == 2, "Class2"] = 1

df["Gender"] = 0
df.loc[df["Sex"] == "male", "Gender"] = 1

df["Age"] = df["Age"]/df["Age"].max()

df["Fare"] = np.log(df["Fare"])

df["SibSp1"] = 0
df.loc[df["SibSp"] == 1, "SibSp1"] = 1

df["SibSp2"] = 0
df.loc[df["SibSp"] == 2, "SibSp2"] = 1

df['Parch1'] = 0
df['Parch2'] = 0
df['Parch3'] = 0
df.loc[(df['Parch'] == 1), 'Parch1'] = 1
df.loc[(df['Parch'] == 2), 'Parch2'] = 1
df.loc[(df['Parch'] == 3), 'Parch3'] = 1

# C = Cherbourg, Q = Queenstown, S = Southampton
df['CherbourgPort'] = 0
df['QueenstownPort'] = 0
df.loc[(df['Embarked'] == "C"), 'CherbourgPort'] = 1
df.loc[(df['Embarked'] == "Q"), 'QueenstownPort'] = 1

# clean up residues
df = df.drop(["Pclass", "Sex", "SibSp", "Parch", "Ticket", "Embarked"], axis = 1)

# add bias in design matrix
df["bias"] = 1

# the log creates inf values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()
count = np.isinf(df).values.sum() 
print("It contains " + str(count) + " infinite values") 

It contains 0 infinite values


C:\Users\MarcoNobile\miniconda3\envs\pyt\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [3]:
Y = torch.tensor(df["Survived"].values, dtype=torch.double)
df = df.drop("Survived", axis=1)
X = torch.tensor(df.values, dtype=torch.double)

loss = torch.nn.MSELoss()# torch.nn.BCEWithLogitsLoss() # # CrossEntropyLoss
relu = torch.nn.ReLU()
lr = 0.001

# Set the range boundaries
lower_bound = -0.1
upper_bound = 0.1
hiddenLayerDim = 32

In [6]:
weightsL1 = (torch.rand(df.shape[1], dtype=torch.double) * (upper_bound - lower_bound)) + lower_bound
weightsL1.requires_grad_()
for i in range(50):
    H1 = torch.matmul(X, weightsL1)    
    H1 = relu(H1)
    output = loss(H1, Y)     
    output.backward()
    with torch.no_grad(): 
        weightsL1 -= weightsL1.grad*lr        
    print(f'step={i}; loss={output:.2f}')

step=0; loss=0.69
step=1; loss=0.69
step=2; loss=0.69
step=3; loss=0.69
step=4; loss=0.69
step=5; loss=0.68
step=6; loss=0.68
step=7; loss=0.68
step=8; loss=0.67
step=9; loss=0.67
step=10; loss=0.66
step=11; loss=0.66
step=12; loss=0.65
step=13; loss=0.65
step=14; loss=0.64
step=15; loss=0.64
step=16; loss=0.63
step=17; loss=0.63
step=18; loss=0.62
step=19; loss=0.62
step=20; loss=0.62
step=21; loss=0.61
step=22; loss=0.61
step=23; loss=0.61
step=24; loss=0.61
step=25; loss=0.61
step=26; loss=0.61
step=27; loss=0.61
step=28; loss=0.61
step=29; loss=0.62
step=30; loss=0.62
step=31; loss=0.62
step=32; loss=0.62
step=33; loss=0.63
step=34; loss=0.63
step=35; loss=0.63
step=36; loss=0.64
step=37; loss=0.64
step=38; loss=0.65
step=39; loss=0.65
step=40; loss=0.65
step=41; loss=0.66
step=42; loss=0.66
step=43; loss=0.66
step=44; loss=0.67
step=45; loss=0.67
step=46; loss=0.67
step=47; loss=0.67
step=48; loss=0.67
step=49; loss=0.67


In [5]:
'''
weightsL1 = (torch.rand((df.shape[1], hiddenLayerDim), dtype=torch.double) * (upper_bound - lower_bound)) + lower_bound
weightsL2 = (torch.rand(hiddenLayerDim, dtype=torch.double) * (upper_bound - lower_bound)) + lower_bound

weightsL1.requires_grad_()
weightsL2.requires_grad_()

for i in range(50):
    H1 = torch.matmul(X, weightsL1)
    H1 = relu(H1)
    H2 = torch.matmul(H1, weightsL2)    
    output = loss(H2, Y)     
    output.backward()
    with torch.no_grad(): 
        weightsL1 -= weightsL1.grad*lr
        weightsL2 -= weightsL2.grad*lr
    print(f'step={i}; loss={output:.2f}')

    
# min step=49; loss=0.63
'''

"\nweightsL1 = (torch.rand((df.shape[1], hiddenLayerDim), dtype=torch.double) * (upper_bound - lower_bound)) + lower_bound\nweightsL2 = (torch.rand(hiddenLayerDim, dtype=torch.double) * (upper_bound - lower_bound)) + lower_bound\n\nweightsL1.requires_grad_()\nweightsL2.requires_grad_()\n\nfor i in range(50):\n    H1 = torch.matmul(X, weightsL1)\n    H1 = relu(H1)\n    H2 = torch.matmul(H1, weightsL2)    \n    output = loss(H2, Y)     \n    output.backward()\n    with torch.no_grad(): \n        weightsL1 -= weightsL1.grad*lr\n        weightsL2 -= weightsL2.grad*lr\n    print(f'step={i}; loss={output:.2f}')\n\n    \n# min step=49; loss=0.63\n"

In [ ]:
###############

In [ ]:
pytorch wants to know in our model class what are the tensor params -> wrap 'em in nn.Params class to make pytorch now detect 
those tensor a weights. Pytorch actually does this with all things that might be a layer of a nn -> eg nn.Linear -> this contains
weights so they get registered